# COVID-19 cases

### Load python tools

In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/Users/mhustiles/Desktop/chromedriver')

### CDC testing capacity

In [2]:
tables = pd.read_html('https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov\
%2Fcoronavirus%2F2019-ncov%2Ftesting-in-us.html',  header=0)

In [3]:
tables = pd.DataFrame(tables[0])

In [4]:
tables.columns = tables.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [5]:
tables.rename(columns={"us_public_health_labs": "US Public Health Labs",\
                   "cdc_labs": "CDC Labs", 'date_collected':'Date'}, inplace=True)

In [6]:
tables['Date'] = pd.to_datetime(tables['Date'] + '/2020')

In [7]:
tables.head()

,Date,CDC Labs,US Public Health Labs
0,2020-01-18,4,0
1,2020-01-19,0,0
2,2020-01-20,7,0
3,2020-01-21,3,0
4,2020-01-22,10,0


In [8]:
tables['US Public Health Labs'] = tables['US Public Health Labs'].str.replace('‡','').str.replace('§','').fillna('0')

In [9]:
tables['CDC Labs'] = tables['CDC Labs'].str.replace('§','').str.replace('‡','').fillna('0')

In [10]:
# tables = tables[tables['Date'] <= '2020/03/10']

In [11]:
tables_melt = pd.melt(tables, id_vars=['Date'], value_vars=['US Public Health Labs', 'CDC Labs'],
        var_name='lab_type', value_name='test_count')

In [12]:
tables.to_csv('output/cdc_testing_table.csv')
tables_melt.to_csv('output/cdc_testing_table_melt.csv')
tables_melt.tail(10)

,Date,lab_type,test_count
192,2020-04-18,CDC Labs,17
193,2020-04-19,CDC Labs,1
194,2020-04-20,CDC Labs,0
195,2020-04-21,CDC Labs,39
196,2020-04-22,CDC Labs,6
197,2020-04-23,CDC Labs,31
198,2020-04-24,CDC Labs,19
199,2020-04-25,CDC Labs,18
200,2020-04-26,CDC Labs,0
201,2020-04-27,CDC Labs,0


In [13]:
tables_melt.head()

,Date,lab_type,test_count
0,2020-01-18,US Public Health Labs,0
1,2020-01-19,US Public Health Labs,0
2,2020-01-20,US Public Health Labs,0
3,2020-01-21,US Public Health Labs,0
4,2020-01-22,US Public Health Labs,0


In [14]:
tables_melt.head()

,Date,lab_type,test_count
0,2020-01-18,US Public Health Labs,0
1,2020-01-19,US Public Health Labs,0
2,2020-01-20,US Public Health Labs,0
3,2020-01-21,US Public Health Labs,0
4,2020-01-22,US Public Health Labs,0


In [15]:
chart = alt.Chart(tables_melt).mark_bar(size=10).encode(
    x=alt.X('Date:T', title='Date', axis=alt.Axis(tickCount=10)),
    y=alt.Y('test_count:Q', title=''),
    color=alt.Color('lab_type', legend=alt.Legend(orient="top"), title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'lab_type',
      sort='ascending'
    )
).properties(width=800, title={
      "text": ["CDC: Daily U.S. coronavirus specimens tested, by lab type"], 
      "subtitle": ["Figures from last four days reported as 'pending'"],
      "color": "black",
      "subtitleColor": "gray"})

chart

alt.Chart(...)

In [16]:
chart.save('visualization.png')

In [17]:
chart.save('visualization.svg')

---

In [18]:
url_cases = 'https://www.cdc.gov/coronavirus/2019-ncov/map-data-cases.csv'

In [19]:
cases = pd.read_csv(url_cases, encoding='latin-1')

In [20]:
cases.columns = cases.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [21]:
cases['cases_reported'] = cases['cases_reported'].str.replace('None', '0')

In [22]:
cases.drop(['unnamed:_5', 'unnamed:_6', 'unnamed:_7'], axis=1, inplace=True)

In [23]:
cases.head()

,jurisdiction,range,cases_reported,community_transmission,url
0,Alabama,51 to 100,51,Undetermined,http://www.adph.org/
1,Alaska,6 to 50,6,Undetermined,http://dhss.alaska.gov/Pages/default.aspx
2,American Samoa,None,0,NaN,http://dhss.as/
3,Arizona,6 to 50,27,Undetermined,http://www.azdhs.gov/
4,Arkansas,6 to 50,37,"Yes, defined area(s)",http://www.healthy.arkansas.gov/Pages/default....


---

### Johns Hopkins data

In [24]:
# Cases: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1

# Deaths: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/0

# Cases countries: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2

# Cases/time: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/cases_time_v3/FeatureServer/0

---

In [25]:
# !tippecanoe --generate-ids --force -r1 -pk -pf -o \
# output/icus_geo.mbtiles \
# output/icus_geo.geojson